In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import optimizers
from keras.optimizers import Adadelta,Adam


#from keras.preprocessing import sequence
from keras.losses import categorical_crossentropy


from google.colab import drive
drive.mount('/content/gdrive')



In [0]:
with open("fer2013.csv") as f:
  content = f.readlines()
  lines = np.array(content)

  x_train, y_train, x_test, y_test = [], [], [], []
  x_val, y_val = [], []
 
  for i in range(1,lines.size):
    try:
      emotion_label, img_data, usage_type = lines[i].split(",")
      pxl_val = img_data.split(" ")
      pixels = np.array(pxl_val, 'float32') 
      emotion_label = keras.utils.to_categorical(emotion_label, num_classes)
      
      if 'Training' in usage_type:
        y_train.append(emotion_label)
        x_train.append(pixels)
      elif 'PublicTest' in usage_type:
        y_test.append(emotion_label)
        x_test.append(pixels)
      elif 'PrivateTest' in usage_type:
    except:
      print("", end="")


x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(28709, 2304)
(3589, 2304)
(28709, 7)
(3589, 7)
(28709, 48, 48, 1)
(3589, 48, 48, 1)
(28709, 7)
(3589, 7)


In [0]:
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras.optimizers import Adadelta

img_rows, img_cols = 48, 48
num_classes = 7
model = Sequential()

#convolution layers
model.add(Conv2D(12, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Conv2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

#model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(AveragePooling2D(pool_size=(3,3)`, strides=(2, 2)))

#model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(Conv2D(16, (3, 3), activation='relu'))
#model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected layers
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(num_classes, activation='softmax'))

ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-07)
model.compile(loss='categorical_crossentropy',
              optimizer=ada,
              metrics=['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 44, 44, 12)        312       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 22, 22, 12)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 18, 18, 32)        9632      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 9, 9, 32)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 2592)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 2048)              5310464   
_________________________________________________________________
dropout_25 (Dropout)         (None, 2048)              0         
__________

In [0]:
    batch_size=300
    epochs=180 #changed from 30 to 70
    model.fit(np.array(x_train), np.array(y_train), batch_size=batch_size,epochs=epochs,validation_split=0.1,verbose=1)
    scores = model.evaluate(np.array(x_test), np.array(y_test), batch_size=batch_size)
    print("Loss: " + str(scores[0]))
    print("Accuracy: " + str(scores[1]))

Train on 25838 samples, validate on 2871 samples
Epoch 1/180
25838/25838 [==============================] - 3s 119us/step - loss: 1.5238 - acc: 0.4159 - val_loss: 1.4775 - val_acc: 0.4385
Epoch 2/180
25838/25838 [==============================] - 3s 114us/step - loss: 1.5069 - acc: 0.4228 - val_loss: 1.4706 - val_acc: 0.4424
Epoch 3/180
25838/25838 [==============================] - 3s 115us/step - loss: 1.4945 - acc: 0.4262 - val_loss: 1.5091 - val_acc: 0.4256
Epoch 4/180
25838/25838 [==============================] - 3s 116us/step - loss: 1.4787 - acc: 0.4332 - val_loss: 1.4582 - val_acc: 0.4364
Epoch 5/180
25838/25838 [==============================] - 3s 115us/step - loss: 1.4642 - acc: 0.4389 - val_loss: 1.4497 - val_acc: 0.4357
Epoch 6/180
25838/25838 [==============================] - 3s 115us/step - loss: 1.4517 - acc: 0.4404 - val_loss: 1.4139 - val_acc: 0.4552
Epoch 7/180
25838/25838 [==============================] - 3s 116us/step - loss: 1.4384 - acc: 0.4477 - val_loss: 1.4

In [0]:
model.save("model_0p56.h5")